=============================================================

#### Explain script work

1 - Use selenium libraly with anti-bot technical included

as exploed the yellowpages website, it have the anti bot for webscraping , so I need to implement the humen similate to avoid each bot detech

- Random delays (human-like behavior)
- User-Agent rotation
- Random screen/viewport sizes
- Session rotation
- Multi-agent pool (parallel browsers)
- Mouse movement simulation
- Request fingerprint randomization
- Stealth mode with selenium-stealth (Python 3.14 compatible)

2 - Go business categories list page and go page by page to get present information

the listing page is showibg basic information, so I extract url to business detail page and basic information , the get another information like phone,address from detail page instend

- 1 Get total shop count from search results
- 2 Count items per page
- 3 Calculate total pages to scrape
- 4 Extract basic data from each listing (name, URL, category)

3 - Visit each shop URL in parallel to get detailed info

regrading to alot of shop page need to visit, i implement multitheard working to visit each page in parraler instend do one by one to handel more information

- 1 Extract address
- 2 Extract phone number(s)
- 4 Handle errors gracefully

4 - Create clean dataframe

=============================================================

=============================================================
#### Workflow instrution
Install required packages

- selenium, webdriver-manager, pandas, openpyxl, tqdm
- fake-useragent (User-Agent rotation)
- selenium-stealth (fingerprint protection)
- Import required libraries

Configure anti-bot settings

- 3.1 Setup User-Agent rotator
- 3.2 Define random screen sizes (11 resolutions)
- 3.3 Configure human-like delays
- 3.4 Setup session rotation settings

Setup human-like behavior functions

- 4.1 Random delay generator
- 4.2 Human-like scrolling (random amounts, pauses)
- 4.3 Mouse movement simulation
- 4.4 Random mouse movements

Setup stealth webdriver

- 5.1 Random screen size selection
- 5.2 Anti-detection Chrome options
- 5.3 Random User-Agent injection
- 5.4 Apply selenium-stealth fingerprint protection
- 5.5 Inject advanced JavaScript stealth (CDP)
- 5.6 Random WebGL/hardware fingerprint

Setup bot detection & handling

- 6.1 Detect Cloudflare challenges
- 6.2 Detect general bot blocks (403, CAPTCHA, rate limit)
- 6.3 Safe navigation with auto-retry

Get shop listings with anti-bot measures

- 7.1 Navigate to search URL with stealth driver
- 7.2 Wait for Cloudflare if detected
- 7.3 Simulate human behavior (scroll, mouse move)
- 7.4 Get total shop count from search results
- 7.5 Count items per page
- 7.6 Calculate total pages to scrape
- 7.7 Extract basic data from each listing (name, URL, address, description, category, Google Maps)
- 7.8 Handle pagination with human-like delays
- 7.9 Rotate session after N requests


Visit each shop URL in parallel with multi-agent pool

- 8.1 Initialize agent pool (multiple browsers with different fingerprints)
- 8.2 Extract phone number(s)
- 8.3 Extract detailed address
- 8.4 Human-like behavior on each page
- 8.5 Auto-rotate agents after N requests
- 8.6 Handle errors gracefully with retry


Create clean dataframe & export

- 9.1 Combine all collected data
- 9.2 Reorder columns
- 9.3 Export to CSV (UTF-8 with BOM for Thai)

=============================================================

## 1. Install Packages and nessesory need

In [1]:
import subprocess

def check_chrome_installed():
    """Checks if Google Chrome is installed and returns its version."""
    try:
        # Attempt to run google-chrome --version
        result = subprocess.run(['google-chrome', '--version'], capture_output=True, text=True, check=True)
        version_output = result.stdout.strip()
        print(f"Google Chrome is already installed: {version_output}")
        return True
    except (subprocess.CalledProcessError, FileNotFoundError):
        print("Google Chrome is not installed.")
        return False

if not check_chrome_installed():
    print("Downloading Google Chrome .deb package...")
    !wget -q -O google-chrome-stable_current_amd64.deb https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
    print("Download complete.")

    print("Installing Google Chrome...")
    !sudo dpkg -i google-chrome-stable_current_amd64.deb
    print("Installation attempt complete. Note: dependency issues might be resolved in the next step.")

    print("Attempting to fix broken dependencies and complete Google Chrome installation...")
    !sudo apt-get install -f -y
    print("Dependency resolution and installation attempt complete.")

print("Verifying Google Chrome installation...")
!google-chrome --version
print("Verification complete.")

Google Chrome is not installed.
Download complete.
Installing Google Chrome...
Selecting previously unselected package google-chrome-stable.
(Reading database ... 121713 files and directories currently installed.)
Preparing to unpack google-chrome-stable_current_amd64.deb ...
Unpacking google-chrome-stable (143.0.7499.40-1) ...
dpkg: dependency problems prevent configuration of google-chrome-stable:
 google-chrome-stable depends on libatk-bridge2.0-0 (>= 2.5.3); however:
  Package libatk-bridge2.0-0 is not installed.
 google-chrome-stable depends on libatk1.0-0 (>= 2.11.90); however:
  Package libatk1.0-0 is not installed.
 google-chrome-stable depends on libatspi2.0-0 (>= 2.9.90); however:
  Package libatspi2.0-0 is not installed.
 google-chrome-stable depends on libvulkan1; however:
  Package libvulkan1 is not installed.
 google-chrome-stable depends on libxcomposite1 (>= 1:0.4.4-1); however:
  Package libxcomposite1 is not installed.

dpkg: error processing package google-chrome-sta

In [2]:
import sys
import subprocess

def install_package(package_name, import_name=None):
    try:
        __import__(import_name or package_name.replace('-', '_'))
        print(f"[OK] {package_name}")
    except ImportError:
        print(f"Installing {package_name}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package_name, "-q"])

# Using regular Selenium + selenium-stealth (works on Python 3.14)
install_package('selenium')
install_package('webdriver-manager')
install_package('pandas')
install_package('openpyxl')
install_package('tqdm')
install_package('fake-useragent', 'fake_useragent')
install_package('selenium-stealth', 'selenium_stealth')
install_package('chromedriver_autoinstaller')
print("\nAll packages ready!")

Installing selenium...
Installing webdriver-manager...
[OK] pandas
[OK] openpyxl
[OK] tqdm
Installing fake-useragent...
Installing selenium-stealth...
Installing chromedriver_autoinstaller...

All packages ready!


## 2. Import Libraries

In [3]:
# Using regular Selenium with stealth
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium_stealth import stealth  # Additional stealth layer
from fake_useragent import UserAgent
import pandas as pd
import time
import random
import urllib.parse
import re
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading

import chromedriver_autoinstaller

In [4]:
def get_chromedriver_path():
    """Auto-download and get the path of ChromeDriver."""
    chromedriver_path = chromedriver_autoinstaller.install()  # This will download the appropriate version if it's not already present
    return chromedriver_path

## 3. Anti-Bot Configuration


In [5]:
# ============================================================================
# ANTI-BOT CONFIGURATION
# ============================================================================
# This section configures various anti-detection techniques to avoid being
# blocked by website protection systems (like Cloudflare, bot detectors, etc.)
# ============================================================================


# 1. USER AGENT ROTATOR
# ----------------------------------------------------------------------------
# Creates a User-Agent generator that provides random browser identities
# This makes each request appear to come from different browsers/devices
ua = UserAgent()


# 2. SCREEN RESOLUTIONS
# ----------------------------------------------------------------------------
# List of common screen resolutions to randomize browser window size
# Real users have different screen sizes, so we mimic this behavior
# Format: (width, height) in pixels
SCREEN_SIZES = [
    (1920, 1080),  # Full HD - Most common
    (1366, 768),   # HD - Laptops
    (1536, 864),   # HD+ - Laptops
    (1440, 900),   # WXGA+ - Older displays
    (1280, 720),   # HD - Small laptops
    (1600, 900),   # HD+ - Monitors
    (1280, 800),   # WXGA - MacBook
    (1024, 768),   # XGA - Tablets
    (1680, 1050),  # WSXGA+ - Widescreen
    (1920, 1200),  # WUXGA - Professional
    (2560, 1440)   # QHD - High-end
]


# 3. HUMAN-LIKE DELAY CONFIGURATION
# ----------------------------------------------------------------------------
# Defines realistic time delays (in seconds) for different actions
# Bots are detected by their unnaturally fast and consistent timing
# Real humans have variable delays between actions
DELAY_CONFIG = {
    'page_load': (2, 5),           # Wait 2-5 seconds after page loads (reading time)
    'between_actions': (0.5, 2),   # Wait 0.5-2 seconds between clicks/scrolls
    'typing': (0.05, 0.15),        # Wait 0.05-0.15 seconds between keystrokes
    'scroll': (0.3, 1),            # Wait 0.3-1 second between scroll movements
    'next_page': (3, 7),           # Wait 3-7 seconds before clicking next page
}


# 4. SESSION ROTATION SETTINGS
# ----------------------------------------------------------------------------
# Controls when to create a new browser session with fresh fingerprint
# Rotating sessions prevents pattern detection from too many requests
SESSION_ROTATION = {
    'enabled': True,                    # Turn session rotation on/off
    'requests_before_rotate': 50,       # Create new session after 50 requests
}


## 4. Human-Like Behavior Functions

In [6]:
# ============================================================================
# HUMAN-LIKE BEHAVIOR SIMULATION FUNCTIONS
# ============================================================================
# Simulate natural human behavior when browsing websites
# Bot detection systems look for robotic patterns - these functions help avoid that
# ============================================================================


def random_delay(delay_type='between_actions'):
    """
    Add random delay to mimic human behavior

    Args:
        delay_type (str): Type of delay from DELAY_CONFIG
                         Options: 'page_load', 'between_actions', 'typing',
                                  'scroll', 'next_page'

    Returns:
        float: The actual delay time used (in seconds)

    Example:
        random_delay('page_load')  # Waits 2-5 seconds randomly
    """
    # Get min and max delay for this action type
    min_delay, max_delay = DELAY_CONFIG.get(delay_type, (1, 3))

    # Generate random delay within range
    delay = random.uniform(min_delay, max_delay)

    # Actually wait for that amount of time
    time.sleep(delay)

    return delay


def human_scroll(driver, scroll_pause_time=None):
    """
    Scroll page like a human - random amounts with random pauses

    Args:
        driver: Selenium WebDriver instance
        scroll_pause_time (float): Base pause time between scrolls (randomized)
    """

    # Real humans don't scroll smoothly to the bottom. They:
    # - Scroll in chunks (not all at once)
    # - Pause to read content
    # - Sometimes pause longer (interesting content)
    # - Vary their scroll speed

    # Set default pause time if not provided
    if scroll_pause_time is None:
        scroll_pause_time = random.uniform(0.3, 1.5)

    # Get total page height and visible viewport height
    total_height = driver.execute_script("return document.body.scrollHeight")
    viewport_height = driver.execute_script("return window.innerHeight")
    current_position = 0

    # Scroll in chunks until reach the bottom
    while current_position < total_height:
        # Random scroll amount: 30-70% of viewport (humans don't scroll exact amounts)
        scroll_amount = random.randint(
            int(viewport_height * 0.3),  # Minimum: 30% of screen
            int(viewport_height * 0.7)   # Maximum: 70% of screen
        )
        current_position += scroll_amount

        # Smooth scroll to new position (looks more natural)
        driver.execute_script(
            f"window.scrollTo({{top: {current_position}, behavior: 'smooth'}});"
        )

        # Pause briefly (reading time)
        time.sleep(random.uniform(0.2, scroll_pause_time))

        # 20% chance to pause longer (simulates reading interesting content)
        if random.random() < 0.2:
            time.sleep(random.uniform(1, 3))


def simulate_mouse_movement(driver, element=None):
    """
    Simulate natural mouse movement

    Bots often don't move the mouse at all, or move it in perfectly straight lines.
    This function adds realistic mouse movement patterns.

    Args:
        driver: Selenium WebDriver instance
        element: Optional - specific element to move mouse towards
    """
    actions = ActionChains(driver)

    if element:
        # Move to element with slight random offset (humans don't click exact center)
        offset_x = random.randint(-5, 5)  # Random horizontal offset
        offset_y = random.randint(-5, 5)  # Random vertical offset
        actions.move_to_element_with_offset(element, offset_x, offset_y)
    else:
        # Random movement somewhere on the page
        viewport_width = driver.execute_script("return window.innerWidth")
        viewport_height = driver.execute_script("return window.innerHeight")

        # Pick random position (avoiding edges)
        x = random.randint(100, viewport_width - 100)
        y = random.randint(100, viewport_height - 100)
        actions.move_by_offset(x, y)

    # Execute the movement
    actions.perform()

    # Brief pause after movement
    time.sleep(random.uniform(0.1, 0.3))


def random_mouse_movements(driver, count=3):
    """
    Perform multiple random mouse movements to appear human

    Args:
        driver: Selenium WebDriver instance
        count (int): Number of random movements to make
    """
    for _ in range(count):
        try:
            # Get viewport dimensions
            viewport_width = driver.execute_script("return window.innerWidth")
            viewport_height = driver.execute_script("return window.innerHeight")

            # Create action chain and move to random position
            actions = ActionChains(driver)
            actions.move_by_offset(
                random.randint(-50, 50),  # Small random movements
                random.randint(-50, 50)
            ).perform()

            # Brief pause between movements
            time.sleep(random.uniform(0.1, 0.3))
        except:
            # Silently ignore errors (element might not be interactable)
            pass


## 5. Stealth Driver Setup

In [7]:
# ============================================================================
# STEALTH WEBDRIVER SETUP
# ============================================================================
# Creates a Chrome browser that's extremely difficult to detect
# as a bot. It uses multiple layers of protection and fingerprint randomization.
# ============================================================================


# Thread-safe counter to track requests across multiple browser instances
request_counter = {'count': 0}
request_lock = threading.Lock()  # Prevents race conditions in multi-threading

# WebGL configurations for hardware fingerprint randomization
# Different computers have different graphics cards - we simulate this
WEBGL_CONFIGS = [
    {'vendor': 'Intel Inc.', 'renderer': 'Intel Iris OpenGL Engine'},
    {'vendor': 'Intel Inc.', 'renderer': 'Intel UHD Graphics 620'},
    {'vendor': 'Intel Inc.', 'renderer': 'Intel HD Graphics 630'},
    {'vendor': 'Intel Inc.', 'renderer': 'Intel Iris Plus Graphics 640'},
    {'vendor': 'Intel Inc.', 'renderer': 'Intel Iris Xe Graphics'},

    {'vendor': 'NVIDIA Corporation', 'renderer': 'GeForce GTX 1080/PCIe/SSE2'},
    {'vendor': 'NVIDIA Corporation', 'renderer': 'GeForce RTX 2080/PCIe/SSE2'},
    {'vendor': 'NVIDIA Corporation', 'renderer': 'GeForce RTX 3060/PCIe/SSE2'},
    {'vendor': 'NVIDIA Corporation', 'renderer': 'GeForce GTX 1050/PCIe/SSE2'},
    {'vendor': 'NVIDIA Corporation', 'renderer': 'Quadro P4000/PCIe/SSE2'},

    {'vendor': 'Advanced Micro Devices, Inc.', 'renderer': 'Radeon RX 580'},
    {'vendor': 'Advanced Micro Devices, Inc.', 'renderer': 'Radeon RX 5700 XT'},
    {'vendor': 'Advanced Micro Devices, Inc.', 'renderer': 'Radeon Pro 560X'},
    {'vendor': 'Advanced Micro Devices, Inc.', 'renderer': 'Radeon Vega 8'},
    {'vendor': 'Advanced Micro Devices, Inc.', 'renderer': 'Radeon RX 6800 XT'},

    {'vendor': 'Apple Inc.', 'renderer': 'Apple M1 GPU'},
    {'vendor': 'Apple Inc.', 'renderer': 'Apple M2 GPU'},
    {'vendor': 'Apple Inc.', 'renderer': 'Apple A14 GPU'},
    {'vendor': 'Apple Inc.', 'renderer': 'Apple Metal Engine'},

    {'vendor': 'Google Inc. (NVIDIA)', 'renderer': 'ANGLE (NVIDIA GeForce GTX 1080)'},
    {'vendor': 'Google Inc. (NVIDIA)', 'renderer': 'ANGLE (NVIDIA GeForce RTX 2080)'},
    {'vendor': 'Google Inc. (NVIDIA)', 'renderer': 'ANGLE (NVIDIA GeForce GTX 1050)'},

    {'vendor': 'Google Inc. (Intel)', 'renderer': 'ANGLE (Intel HD Graphics 630)'},
    {'vendor': 'Google Inc. (Intel)', 'renderer': 'ANGLE (Intel Iris Plus Graphics 640)'},
    {'vendor': 'Google Inc. (Intel)', 'renderer': 'ANGLE (Intel Iris Xe Graphics)'},

    {'vendor': 'Google Inc. (AMD)', 'renderer': 'ANGLE (AMD Radeon RX 580)'},
    {'vendor': 'Google Inc. (AMD)', 'renderer': 'ANGLE (AMD Radeon Pro 560X)'},
    {'vendor': 'Google Inc. (AMD)', 'renderer': 'ANGLE (AMD Radeon Vega 8)'},
]



def setup_driver(headless=True):
    """
    Setup Chrome driver with MAXIMUM anti-bot protection

    This function creates a browser instance that:
    1. Hides automation indicators
    2. Randomizes browser fingerprints
    3. Injects stealth JavaScript
    4. Mimics real user hardware/software

    Args:
        headless (bool): If True, browser runs without visible window

    Returns:
        tuple: (driver, user_agent) - WebDriver instance and its User-Agent string
    """

    # Initialize Chrome options
    options = Options()

    # STEP 1: RANDOMIZE SCREEN SIZE
    # -------------------------------------------------------------------------
    # Pick a random screen resolution from our list
    # This makes each browser session look like a different device
    width, height = random.choice(SCREEN_SIZES)
    options.add_argument(f'--window-size={width},{height}')

    # STEP 2: HEADLESS MODE (OPTIONAL)
    # -------------------------------------------------------------------------
    # Run browser without GUI (faster, uses less resources)
    # 'new' headless mode is harder to detect than old headless
    if headless:
        options.add_argument('--headless=new')


    # STEP 3: CORE ANTI-DETECTION OPTIONS
    # -------------------------------------------------------------------------
    # These are the most important options to hide automation

    # Remove 'navigator.webdriver' flag (biggest bot indicator)
    options.add_argument('--disable-blink-features=AutomationControlled')

    # Hide automation-related switches and logging
    options.add_experimental_option('excludeSwitches', ['enable-automation', 'enable-logging'])

    # Disable automation extension
    options.add_experimental_option('useAutomationExtension', False)


    # STEP 4: BROWSER FINGERPRINT RANDOMIZATION
    # -------------------------------------------------------------------------
    # Make each session look like a different user/device

    # Random User-Agent (browser identity)
    user_agent = ua.random
    options.add_argument(f'--user-agent={user_agent}')

    # Random language preference (English or Thai)
    options.add_argument(f'--lang={random.choice(["en-US,en", "th-TH,th,en", "en-GB,en"])}')


    # STEP 5: STABILITY & PERFORMANCE OPTIONS
    # -------------------------------------------------------------------------
    # These prevent crashes and reduce resource usage

    options.add_argument('--disable-dev-shm-usage')  # Overcome limited resource problems
    options.add_argument('--no-sandbox')             # Bypass OS security model
    options.add_argument('--disable-gpu')            # Disable GPU hardware acceleration
    options.add_argument('--disable-extensions')     # Disable browser extensions
    options.add_argument('--disable-infobars')       # Disable info bars
    options.add_argument('--disable-notifications')  # Block notification popups
    options.add_argument('--disable-popup-blocking') # Allow popups (some sites need this)

    # Reduce console noise
    options.add_argument('--disable-logging')
    options.add_argument('--log-level=3')  # Only show fatal errors
    options.add_argument('--silent')


    # STEP 6: CREATE DRIVER INSTANCE
    # -------------------------------------------------------------------------
    # Initialize Chrome with all our configured options
    #service = Service(ChromeDriverManager().install())
    service = Service(get_chromedriver_path())
    driver = webdriver.Chrome(service=service, options=options)


    # STEP 7: APPLY SELENIUM-STEALTH LIBRARY
    # -------------------------------------------------------------------------
    # Additional layer of protection using selenium-stealth package

    # Pick random WebGL configuration (graphics card fingerprint)
    webgl = random.choice(WEBGL_CONFIGS)

    # Apply stealth patches
    stealth(driver,
        languages=['th-TH', 'th', 'en-US', 'en'],  # Language preferences
        vendor=webgl['vendor'],                     # GPU vendor
        platform='Win32',                           # Operating system
        webgl_vendor=webgl['vendor'],               # WebGL vendor
        renderer=webgl['renderer'],                 # GPU renderer
        fix_hairline=True,                          # Fix hairline rendering
    )


    # STEP 8: ADVANCED JAVASCRIPT STEALTH INJECTION
    # -------------------------------------------------------------------------
    # Inject custom JavaScript to hide automation traces
    # This is the most advanced protection layer

    # Randomize hardware specs
    hw_cores = random.choice([4, 6, 8, 12, 16])  # CPU cores
    dev_mem = random.choice([4, 8, 16])          # RAM in GB
    rtt = random.randint(50, 150)                # Network latency (ms)

    # JavaScript code to inject into every page
    stealth_js = f'''
        // HIDE WEBDRIVER FLAG
        // Remove the 'navigator.webdriver' property that screams "I'm a bot!"
        Object.defineProperty(navigator, 'webdriver', {{get: () => undefined}});

        // FAKE BROWSER PLUGINS
        // Real browsers have plugins, bots often don't
        Object.defineProperty(navigator, 'plugins', {{get: () => [1,2,3,4,5]}});

        // SET REALISTIC LANGUAGES
        // Match our User-Agent language settings
        Object.defineProperty(navigator, 'languages', {{get: () => ['th-TH','th','en-US','en']}});

        // RANDOMIZE CPU CORES
        // Different computers have different CPU counts
        Object.defineProperty(navigator, 'hardwareConcurrency', {{get: () => {hw_cores}}});

        // RANDOMIZE DEVICE MEMORY
        // Simulate different RAM amounts
        Object.defineProperty(navigator, 'deviceMemory', {{get: () => {dev_mem}}});

        // SET PLATFORM
        // Report as Windows
        Object.defineProperty(navigator, 'platform', {{get: () => 'Win32'}});

        // ADD CHROME-SPECIFIC OBJECTS
        // Headless Chrome often lacks these objects
        window.chrome = {{runtime: {{}}, loadTimes: function(){{}}, csi: function(){{}}}};

        // FAKE NETWORK CONNECTION INFO
        // Real browsers expose network information
        Object.defineProperty(navigator, 'connection', {{
            get: () => ({{
                effectiveType: '4g',    // Connection type
                rtt: {rtt},             // Round-trip time (latency)
                downlink: 10,           // Download speed (Mbps)
                saveData: false         // Data saver mode
            }})
        }});
    '''

    # Try to inject via Chrome DevTools Protocol (preferred method)
    try:
        driver.execute_cdp_cmd('Page.addScriptToEvaluateOnNewDocument', {'source': stealth_js})
    except:
        # Fallback: inject directly (less reliable)
        try:
            driver.execute_script(stealth_js)
        except:
            pass  # If both fail, continue anyway


    # STEP 9: SET FINAL WINDOW SIZE
    # -------------------------------------------------------------------------
    driver.set_window_size(width, height)

    return driver, user_agent


def should_rotate_session():
    """
    Check if we should create a new browser session

    Thread-safe counter that tracks total requests across all browsers.
    After N requests, we rotate to a fresh session with new fingerprint.

    Returns:
        bool: True if it's time to rotate, False otherwise
    """
    # Check if rotation is enabled
    if not SESSION_ROTATION['enabled']:
        return False

    # Thread-safe counter increment
    with request_lock:
        request_counter['count'] += 1

        # Check if we've hit the rotation threshold
        if request_counter['count'] >= SESSION_ROTATION['requests_before_rotate']:
            request_counter['count'] = 0  # Reset counter
            return True

    return False



def detect_cloudflare(driver):
    """
    Detect if Cloudflare challenge is present

    Cloudflare is a popular bot protection service that shows
    a "Checking your browser" page before allowing access.

    Args:
        driver: Selenium WebDriver instance

    Returns:
        bool: True if Cloudflare detected, False otherwise
    """
    page_source = driver.page_source.lower()

    # Common Cloudflare indicators
    cloudflare_indicators = [
        'checking your browser',   # Main challenge message
        'cloudflare',              # Cloudflare branding
        'please wait',             # Challenge in progress
        'ddos protection',         # DDoS protection message
        'ray id',                  # Cloudflare Ray ID
        'please turn javascript on',  # JS requirement
        'enable cookies'           # Cookie requirement
    ]

    return any(indicator in page_source for indicator in cloudflare_indicators)


def wait_for_cloudflare(driver, timeout=30):
    """
    Wait for Cloudflare challenge to complete

    Cloudflare challenges usually complete automatically after 5-10 seconds.
    This function waits for the challenge to finish.

    Args:
        driver: Selenium WebDriver instance
        timeout (int): Maximum seconds to wait

    Returns:
        bool: True if challenge completed, False if timeout
    """
    start_time = time.time()

    # Keep checking until challenge is gone or timeout
    while detect_cloudflare(driver):
        # Check if we've exceeded timeout
        if time.time() - start_time > timeout:
            print("Warning: Cloudflare challenge timeout")
            return False

        # Wait a bit before checking again
        time.sleep(2)

    return True  # Challenge completed!


## 6. URL Collection with Anti-Bot

In [8]:
# ============================================================================
# URL COLLECTION WITH ANTI-BOT PROTECTION
# ============================================================================
# Collects all business listing URLs from Yellow Pages search results
# It handles pagination, anti-bot detection, and session rotation automatically
# ============================================================================


def collect_urls(keyword, max_pages=None):
    """Collect business URLs from Yellow Pages"""
    driver, _ = setup_driver(headless=True)
    all_businesses = []

    try:
        url = f'https://www.yellowpages.co.th/heading/{urllib.parse.quote(keyword)}'
        print(f"\nSearching: {keyword}\n")

        driver.get(url)
        random_delay('page_load')

        # checl for cloundflare anti bot blocked
        if detect_cloudflare(driver):
            wait_for_cloudflare(driver)

        # simulate humen behabior
        random_mouse_movements(driver, count=2)
        human_scroll(driver)

        # Calculate total pages
        total_pages = max_pages
        try:
            # extract number of business result
            result_text = driver.find_elements(By.XPATH, '//*[contains(text(), "ผลการค้นหา")]')[0].text
            total_results = int(re.findall(r'[\d,]+', result_text)[0].replace(',', ''))

            # Check for number of shop on first page
            items_per_page = len(driver.find_elements(By.CSS_SELECTOR, 'div.yp-search-listing'))
            total_pages = min((total_results + items_per_page - 1) // items_per_page, max_pages or 999)
            print(f"Found {total_results:,} results, scraping {total_pages} pages\n")
        except:
            pass

        page = 1
        with tqdm(total=total_pages, desc='Collecting', unit='page') as pbar:
            while True:
                if max_pages and page > max_pages:
                    break

                # Session rotation for avoid anti bot
                if should_rotate_session():
                    driver.quit()
                    driver, _ = setup_driver(headless=True)
                    driver.get(f'{url}?page={page}' if page > 1 else url)
                    random_delay('page_load')

                # Get listings
                listings = WebDriverWait(driver, 15).until(
                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.yp-search-listing'))
                )
                human_scroll(driver)

                # Extract data from listing includ ebusiness url , name , description , categories, category , google map url
                for listing in listings:
                    try:
                        link = listing.find_element(By.CSS_SELECTOR, 'a[data-businessid]')
                        business = {
                            'url': link.get_attribute('href'),
                            'listing_name': link.text.strip(),
                            'listing_description': '',
                            'listing_category': '',
                            'google_maps_url': ''
                        }

                        # Extract description
                        try:
                            business['listing_description'] = listing.find_element(By.CSS_SELECTOR, 'p.yp-listing-desc').text.strip()
                        except: pass
                        # Extract category
                        try:
                            business['listing_category'] = listing.find_element(By.CSS_SELECTOR, 'div.listing-category-section a').text.strip()
                        except: pass
                        # Extract google map url
                        try:
                            business['google_maps_url'] = listing.find_element(By.CSS_SELECTOR, 'a.icon-navigate[href*="maps.google.com"]').get_attribute('href')
                        except: pass

                        if business['url'] and business['url'] not in [b['url'] for b in all_businesses]:
                            all_businesses.append(business)
                    except:
                        pass

                pbar.update(1)

                # Next page
                try:
                    next_btn = driver.find_element(By.CSS_SELECTOR, 'a[rel="next"]')
                    if not next_btn.is_displayed():
                        break
                    simulate_mouse_movement(driver, next_btn)
                    random_delay('between_actions')
                    driver.execute_script("arguments[0].click();", next_btn)
                    random_delay('next_page')
                    page += 1
                except:
                    break

        print(f"\nCollected {len(all_businesses)} businesses\n")
        return all_businesses

    finally:
        driver.quit()


## 7. Detail Scraping with Anti-Bot

In [9]:
# ============================================================================
# BUSINESS DETAIL EXTRACTION
# ============================================================================

def scrape_details(businesses, max_workers=3, max_retries=2, debug=False):
    """Scrape business details in parallel"""
    print(f"\nExtracting details from {len(businesses)} businesses...\n")

    def process_business(business):
        driver, _ = setup_driver(headless=True)
        try:
            # Extract with retry logic
            for attempt in range(max_retries):
                try:
                    driver.get(business['url'])
                    random_delay('page_load')
                    random_mouse_movements(driver, count=1)
                    human_scroll(driver)

                    # Extract phones
                    phones = [elem.text.strip() for elem in driver.find_elements(By.CSS_SELECTOR, 'a[href^="tel:"]')
                             if elem.text.strip() and len(elem.text.strip()) >= 9]
                    business['phone'] = ', '.join(list(dict.fromkeys(phones)))

                    # Extract address (supports multiple page types)
                    business['address'] = ''
                    try:
                        # Method 1: YellowPages Profile Page (strong tag with "ที่อยู่")
                        elems = driver.find_elements(By.XPATH, '//strong[contains(text(), "ที่อยู่")]/parent::div/following-sibling::div')
                        if elems and elems[0].text.strip():
                            business['address'] = elems[0].text.strip()

                        # Method 2: YellowPages Landing Page (item-address class)
                        if not business['address']:
                            elems = driver.find_elements(By.XPATH, '//li[contains(@class, "item-address")]//span')
                            if elems and elems[0].text.strip():
                                business['address'] = elems[0].text.strip()


                    except:
                        business['address'] = ''

                    # Retry
                    if attempt < max_retries - 1:
                        random_delay('page_load')
                        driver.refresh()
                        random_delay('page_load')

                except Exception as e:
                    if debug:
                        print(f"Error: {str(e)}")
                    if attempt < max_retries - 1:
                        random_delay('page_load')

            random_delay('between_actions')
            return business

        finally:
            driver.quit()

    results = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(process_business, biz): biz for biz in businesses}

        with tqdm(total=len(businesses), desc='Extracting', unit='business') as pbar:
            for future in as_completed(futures):
                try:
                    results.append(future.result())
                except Exception as e:
                    business = futures[future]
                    business.update({'phone': '', 'address': ''})
                    results.append(business)
                    if debug:
                        print(f"Failed: {business.get('listing_name', 'Unknown')}")
                pbar.update(1)

    print(f"\nComplete! Processed {len(results)} businesses\n")
    return results


## 8. Configuration

In [10]:
# Search Configuration
keyword = 'กุญแจ'  # Search keyword
max_pages = None  # Set to None for all pages, or a number to limit (start small for testing)
max_workers = 2  # Number of parallel agents (lower = safer, higher = faster)

# Anti-Bot Settings
DELAY_CONFIG['page_load'] = (6, 12)  # Increase delays for more safety
DELAY_CONFIG['next_page'] = (4, 8)

print("Configuration loaded!")
print(f"  Keyword: {keyword}")
print(f"  Max pages: {max_pages or 'All'}")
print(f"  Agents: {max_workers}")


Configuration loaded!
  Keyword: กุญแจ
  Max pages: All
  Agents: 2


## 9. Run Scraper

In [11]:
# Collect URLs with anti-bot measures
businesses = collect_urls(keyword, max_pages=max_pages)

# Scrape details using multi-agent pool
if businesses:
    data = scrape_details(
        businesses,
        max_workers=max_workers
    )
else:
    print('No businesses found')
    data = []



Searching: กุญแจ

Found 271 results, scraping 28 pages



Collecting: 100%|██████████| 28/28 [08:36<00:00, 18.46s/page]



Collected 270 businesses


Extracting details from 270 businesses...



Extracting: 100%|██████████| 270/270 [2:13:46<00:00, 29.73s/business]


Complete! Processed 270 businesses



## 10. Create DataFrame & Export

In [12]:
if data:
    df = pd.DataFrame(data)

    # Reorder columns
    columns_order = ['listing_name', 'listing_description',
                     'listing_category', 'google_maps_url', 'phone', 'address', 'url']
    df = df[[col for col in columns_order if col in df.columns]]

    print(f"\nDataFrame created with {len(df)} rows")
    display(df.head(10))

    # Export to CSV
    filename = f'yellowpages_{keyword}.csv'
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"\nExported to: {filename}")
else:
    print('No data to export')


DataFrame created with 270 rows


,listing_name,listing_description,listing_category,google_maps_url,phone,address,url
0,ช่างกุญแจรถยนต์ กรุงเทพ - อ.วิทย์กุญแจ,ช่างกุญแจรถยนต์ รับทำทุกรุ่น บริการนอกสถานที่ ...,ช่างกุญแจ,https://maps.google.com/maps/place?q=13.803567...,"081-457-7373, 089-494-5224, 02-933-3246",184 ซอยถนนโชคชัย 4 ระหว่างแยก 14-16 อยู่ตรงกลา...,https://www.carkey184.com/
1,ช่างกุญแจ 24 ชั่วโมง ชลบุรี - คีย์ 24,ช่างกุญแจ 24 ชั่วโมง ชลบุรี หากเจ้าของบ้านลืมก...,ช่างกุญแจ 24 ชม.,https://maps.google.com/maps/place?q=13.037582...,098-799-9349,274/1 หมู่ที่ 7 ตำบลบ่อวิน อำเภอศรีราชา ชลบุรี...,https://www.key24hour.com/
2,ร้านกุญแจกรุงเทพ - Racingkeys,บริการซ่อมกุญแจชิฟ ซ่อมดอกกุญแจหัก เปลี่ยนกรอบ...,ช่างกุญแจ 24 ชม.,https://maps.google.com/maps/place?q=13.848743...,081-487-5437,944 ห้องที่ 3 ซอยรามอินทรา 67 แขวงท่าแร้ง เขตบ...,https://www.yellowpages.co.th/profile/%E0%B8%A...
3,ร้าน รับทำกุญแจ รับซ่อมกุญแจทุกชนิด ปทุมธานี -...,ร้านรับทำกุญแจสำรองเพิ่ม รับปั๊มลูกกุญแจบ้านสำ...,ช่างกุญแจ,https://maps.google.com/maps/place?q=14.053407...,"081-760-6868, 081-696-7332, 08-1760-6868",111/72 หมู่ที่ 4 ถนนปทุมธานี-บางเลน ตำบลลาดหลุ...,https://www.xn--12coo5bfbfn9cn9ni4b7ae4lpf.com/
4,เจ กุญแจเชียงใหม่,รับทำกุญแจรถมอเตอร์ไซค์ทุกรุ่นทุกยี่ห้อ รับปั๊...,ช่างกุญแจ 24 ชม.,"https://maps.google.com/maps/place?q=18.77352,...","089-851-2309, 089-435-0903, 084-378-3185",3/1 ถนนศรีปิงเมือง ตำบลช้างคลาน อำเภอเมืองเชีย...,https://www.yellowpages.co.th/profile/%E0%B9%8...
5,เชียงใหม่กุญแจ 24 ชม,เรียก ช่างกุญแจ เชียงใหม่ รับปลดล็อคกุญแจรถ กุ...,ช่างกุญแจ 24 ชม.,"https://maps.google.com/maps/place?q=18.76921,...","086-182-5094, 085-035-3766",286 ถนนช้างคลาน ตำบลช้างคลาน อำเภอเมืองเชียงให...,https://www.yellowpages.co.th/profile/%E0%B9%8...
6,ต๋อย กุญแจ เชียงใหม่,"ลืมกุญแจรถ เมืองเชียงใหม่, ทำกุญแจรถหาย เมืองเ...",ช่างกุญแจ,"https://maps.google.com/maps/place?q=18.80946,...",081-883-8584,85/2 ถนนช้างเผือก ตำบลช้างเผือก อำเภอเมืองเชีย...,https://www.yellowpages.co.th/profile/%E0%B8%9...
7,ช่างกุญแจ บ้าน รถ ตู้เซฟ,ช่างนีการกุญแจ รับแก้ปัญหา ลูกกุญแจรถแต่ไขสตาร...,ช่างกุญแจ 24 ชม.,https://maps.google.com/maps/place?q=13.922158...,098-447-7748,121/161 (หมู่บ้านมนชญา 1) ซอยสายไหม แขวงสายไหม...,https://www.yellowpages.co.th/profile/%E0%B8%8...
8,ช่างกุญแจรถ กุญแจบ้าน กุญแจตู้เซฟ,แนะนำช่างกุญแจรถยนต์ดี ๆ ฝั่งธนบุรี มีวิธีแก้ป...,ช่างกุญแจ 24 ชม.,https://maps.google.com/maps/place?q=13.729505...,081-804-2852,396 ซอยเพชรเกษม 63 แขวงหลักสอง เขตบางแค กรุงเท...,https://www.yellowpages.co.th/profile/%E0%B8%8...
9,ช่างกุญแจ สุขุมวิท รับทำกุญแจทุกชนิด,ลืมกุญแจไว้ในรถ ทำกุญแจรถหาย รีโมทกุญแจเสียเข้...,ช่างกุญแจ 24 ชม.,https://maps.google.com/maps/place?q=13.733898...,086-006-1814,398 ถนนสุขุมวิท แขวงคลองเตย เขตคลองเตย กรุงเทพ...,https://www.yellowpages.co.th/profile/%E0%B8%8...



Exported to: yellowpages_กุญแจ.csv
